<a href="https://colab.research.google.com/github/Sakib56/TTDS-G35-CW3/blob/main/python/draft_vector_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Uncomment and run the following cells if you work on GCP. Change runtime type to GPU.

In [2]:
pip install torch==1.8.1 transformers==3.3.1 sentence-transformers==0.3.8 pandas==1.1.2 faiss-cpu==1.6.1 numpy==1.19.2 folium==0.2.1 streamlit==0.62.0

     |████████████████████████████████| 804.1 MB 2.6 kB/s 
     |████████████████████████████████| 1.1 MB 89.0 MB/s 
     |████████████████████████████████| 66 kB 5.8 MB/s 
     |████████████████████████████████| 10.5 MB 47.8 MB/s 
     |████████████████████████████████| 7.1 MB 20.5 MB/s 
     |████████████████████████████████| 14.5 MB 67.9 MB/s 
     |████████████████████████████████| 69 kB 9.6 MB/s 
     |████████████████████████████████| 7.1 MB 36.8 MB/s 
     |████████████████████████████████| 3.0 MB 63.9 MB/s 
     |████████████████████████████████| 895 kB 85.5 MB/s 
     |████████████████████████████████| 1.2 MB 61.7 MB/s 
     |████████████████████████████████| 76 kB 6.0 MB/s 
     |████████████████████████████████| 132 kB 77.8 MB/s 
     |████████████████████████████████| 111 kB 86.5 MB/s 
     |████████████████████████████████| 4.3 MB 61.8 MB/s 
     |████████████████████████████████| 8.6 MB 65.6 MB/s 
     |████████████████████████████████| 138 kB 84.4 MB/s 
     |███████████

In [1]:
pip install --upgrade pandas

     |████████████████████████████████| 11.3 MB 7.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.2
    Uninstalling pandas-1.1.2:
      Successfully uninstalled pandas-1.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.1 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.32.0 which is incompatible.




---
This is mounting my (Kenza) drive to the collab notebook. I stored the wikidata there.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Before we begin, make sure you restart (not factory reset) the runtime so that the relevant packages are used

In [3]:
%load_ext autoreload

In [7]:
%autoreload 2
# Used to import data from local.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle
from pathlib import Path

from nltk.stem import PorterStemmer
import re
ps = PorterStemmer()

In [8]:
def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    query = ps.stem(query)
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]

In [33]:
import xml.etree.ElementTree as ET
context = ET.iterparse('/content/drive/MyDrive/Colab Notebooks/wikidata_short.xml', events=('end', ))
df = pd.DataFrame(columns = ['id', 'title', 'text'])

NSMAP = {'md': 'http://www.mediawiki.org/xml/export-0.10/'}
for event, elem in context:
    if 'page' in elem.tag:
        title = elem.find('title', namespaces=NSMAP).text
        id = elem.find('id', namespaces=NSMAP).text
        text = elem.find('revision').find('text').text
        text = ' '.join([ps.stem(word.strip()) for word in re.split('[^a-zA-Z0-9]', text) if word != '' and word.lower()])
        df = df.append({'id': float(id), 'title': title, 'text': text}, ignore_index=True)
        

In [34]:
df.head()

,id,title,text
0,12.0,Anarchism,short descript polit philosophi and movement r...
1,13.0,AfghanistanHistory,redirect histori of afghanistan redirect categ...
2,14.0,AfghanistanGeography,redirect geographi of afghanistan redirect cat...
3,15.0,AfghanistanPeople,redirect demograph of afghanistan redirect cat...
4,18.0,AfghanistanCommunications,redirect commun in afghanistan redirect catego...


In [20]:
print(f"Wikidata articles: {df.id.unique().shape[0]}")

Wikidata articles: 339


The [Sentence Transformers library](https://github.com/UKPLab/sentence-transformers) offers pretrained transformers that produce SOTA sentence embeddings. Checkout this [spreadsheet](https://docs.google.com/spreadsheets/d/14QplCdTCDwEmTqrn1LH4yrbKvdogK4oQvYO1K1aPR5M/) with all the available models.

In this tutorial, we will use the `distilbert-base-nli-stsb-mean-tokens` model which has the best performance on Semantic Textual Similarity tasks among the DistilBERT versions. Moreover, although it's slightly worse than BERT, it is quite faster thanks to having a smaller size.

In [21]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
# Check if GPU is available and use it
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)

100%|██████████| 245M/245M [00:16<00:00, 15.0MB/s]


cuda:0


In [22]:
# Convert abstracts to vectors
embeddings = model.encode(df.text.to_list(), show_progress_bar=True)

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [23]:
print(f'Shape of the vectorised abstract: {embeddings[0].shape}')

Shape of the vectorised abstract: (768,)


## Vector similarity search with Faiss
[Faiss](https://github.com/facebookresearch/faiss) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, even ones that do not fit in RAM. 
    
Faiss is built around the `Index` object which contains, and sometimes preprocesses, the searchable vectors. Faiss has a large collection of [indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes). You can even create [composite indexes](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes-(composite)). Faiss handles collections of vectors of a fixed dimensionality d, typically a few 10s to 100s.

**Note**: Faiss uses only 32-bit floating point matrices. This means that you will have to change the data type of the input before building the index.

To learn more about Faiss, you can read their paper on [arXiv](https://arxiv.org/abs/1702.08734).

To speed up the search, it is possible to segment the dataset into pieces. We define Voronoi cells in the d-dimensional space, and each database vector falls in one of the cells. At search time, only the database vectors y contained in the cell the query x falls in and a few neighboring ones are compared against the query vector.

This is done via the IndexIVFFlat index. This type of index requires a training stage, that can be performed on any collection of vectors that has the same distribution as the database vectors. In this case we just use the database vectors themselves.

The IndexIVFFlat also requires another index, the quantizer, that assigns vectors to Voronoi cells. Each cell is defined by a centroid, and finding the Voronoi cell a vector falls in consists in finding the nearest neighbor of the vector in the set of centroids. This is the task of the other index, which is typically an IndexFlatL2.

There are two parameters to the search method: nlist, the number of cells, and nprobe, the number of cells (out of nlist) that are visited to perform a search. The search time roughly increases linearly with the number of probes plus some constant due to the quantization.

To create an index with the `wikidata` abstract vectors, we will:
1. Change the data type of the text vectors to float32.
2. Build an index and pass it the dimension of the vectors it will operate on.
3. Pass the index to IndexIDMap, an object that enables us to provide a custom list of IDs for the indexed vectors.
4. Add the abstract vectors and their ID mapping to the index. In our case, we will map vectors to their page IDs from MAG.

In [35]:
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# Step 2: Instantiate the index
quantizer = faiss.IndexFlatL2(embeddings.shape[1])
nlist = 2000
index = faiss.IndexIVFFlat(quantizer, embeddings.shape[1], nlist)

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, np.array(df.id.values))

print(f"Number of vectors in the Faiss index: {index.ntotal}")

TypeError: ignored

In [31]:
type(df.id.values[0]), type(embeddings[0])

(int, numpy.ndarray)

### Searching the index
The index we built will perform a k-nearest-neighbour search. We have to provide the number of neighbours to be returned. 

Let's query the index with an abstract from our dataset and retrieve the 10 most relevant documents. **The first one must be our query!**


In [ ]:
# Wikidata Text
df.iloc[300, 6]

'REDIRECT Lycoming T55 Redirect category shell R from incorrect name R from move'

In [ ]:
# Retrieve the 10 nearest neighbours
D, I = index.search(np.array([embeddings[300]]), k=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [0.0, 124.00892639160156, 127.1615982055664, 131.80374145507812, 132.68504333496094, 142.94412231445312, 143.86862182617188, 146.7510986328125, 148.06011962890625, 149.39564514160156]

MAG paper IDs: [69171661, 42, 69171578, 69171349, 69171555, 69171644, 69171463, 69171442, 69171739, 69171429]


In [ ]:
# Fetch the Wikipedia titles based on their index
id2details(df, I, 'title')

[['Honeywell T55 Turboshaft Engine'],
 ['ArtificalLanguages'],
 ['Draft:Vasudha Dhagamwar'],
 ['Illawarra cattle'],
 ['Masdar Institute of Science and Technology'],
 ['Draft:Jeevan Bahadur Shahi'],
 ['Henri Louit'],
 ["Principauté d'Aigues-Mortes"],
 ['Quan khi Dao'],
 ['Georges Herpin']]

In [ ]:
# Fetch the Wikipedia text based on their index
id2details(df, I, 'text')

[['REDIRECT Lycoming T55 Redirect category shell R from incorrect name R from move'],
 ['REDIRECT Constructed language Rcat shell R from move R from misspelling Artificial languages R from CamelCase Unprintworthy redirect'],
 ['REDIRECT Vasudha Dhagamwar Redirect category shell R from move'],
 ['REDIRECT Illawarra Shorthorn Redirect category shell R from move'],
 ['REDIRECT Masdar Institute Redirect category shell R from move R from shorter name R from alternative name'],
 ['REDIRECT Jeevan Bahadur Shahi Redirect category shell R from move'],
 ['REDIRECT Honor Louit Redirect category shell R from move'],
 ['REDIRECT Principality of Aigues Mortes R from alternative language fr en'],
 ['REDIRECT Qwan Ki Do Rcat shell R from alternative name'],
 ['REDIRECT Guy Herpin Redirect category shell R from move']]


## Putting all together

So far, we've built a Faiss index using the wikidata text vectors we encoded with a sentence-DistilBERT model. That's helpful but in a real case scenario, we would have to work with unseen data. To query the index with an unseen query and retrieve its most relevant documents, we would have to do the following:

1. Encode the stemmed query with the same sentence-DistilBERT model we used for the rest of the abstract vectors.
2. Change its data type to float32.
3. Search the index with the encoded query.

IDEA: Use the Answer of the Question Answering option as the input query for vector search or let the user write a query for vector search or both.


In [ ]:
user_query = """
short description Political philosophy and movement redirect2 Anarchist Anarchists other uses Anarchists disambiguation pp semi indef good article use British English date August 2021 use dmy dates date August 2021 anarchism sidebar basic forms of government Anarchism is a political philosophy and Political movement movement that is sceptical of authority and rejects all involuntary coercive forms o"""

In [ ]:
# For convenience, I've wrapped all steps in the vector_search function.
# It takes four arguments: 
# A query, the sentence-level transformer, the Faiss index and the number of requested results
D, I = vector_search([user_query], model, index, num_results=10)
print(f'L2 distance: {D.flatten().tolist()}\n\nMAG paper IDs: {I.flatten().tolist()}')

L2 distance: [28.52496337890625, 156.78256225585938, 157.7370147705078, 164.12826538085938, 175.03289794921875, 175.9829864501953, 183.06655883789062, 184.00244140625, 186.358642578125, 188.88702392578125]

MAG paper IDs: [12, 69171335, 69171296, 69171737, 69171397, 69171603, 69171636, 25, 69171529, 69171305]


In [ ]:
# Fetching the paper titles based on their index
id2details(df, I, 'title')

[['Anarchism'],
 ['Draft:William W. Stow'],
 ['Draft:Anti-equilibrium theory'],
 ['Draft:Flop flips'],
 ['Draft:AVAN SIMMONS'],
 ['Draft:The Iron Order'],
 ['Draft:San Marcos creek'],
 ['Autism'],
 ['Wikipedia:Meetup/Black Lunch Table/LA photodrive21'],
 ["File:Pancho's Takos logo.png"]]

In [ ]:
# Define project base directory
# Change the index from 1 to 0 if you run this on Google Colab
project_dir = Path('notebooks').resolve().parents[0]
print(project_dir)

# Serialise index and store it as a pickle
with open(f"{project_dir}/models/faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

/content


FileNotFoundError: ignored